# Java

Java is the mother of all JVM languages, first released in 1995 after years of development by Sun Microsystems.  BeakerX uses [OpenJDK](http://openjdk.java.net/) for Java and all the kernels.

BeakerX supports Java cells that define classes, which can be used by other JVM based cells.
One cell is equivalent to a Java compilation unit and can contain a single class, or a sequence of statements.

In [3]:
%classpath add mvn org.json json 20180813

In [12]:
import org.json.JSONObject;
import org.json.JSONArray;

import java.io.FileReader;
import java.io.BufferedReader;

import java.io.FileNotFoundException;
import java.io.IOException;

import java.io.InputStreamReader;
import java.net.URL;
import java.net.URLEncoder;
import java.util.Collection;
import java.util.List;
import java.util.ArrayList;
import java.util.Scanner;

String adr="60, Rue Réaumur";//"28, rue notre dame des champs";
double score= -Double.MAX_VALUE;
double latitude= Double.NaN;
double longitude= Double.NaN;
        URL url= new URL("https://api-adresse.data.gouv.fr/search/?q="+URLEncoder.encode(adr, "UTF-8")+"&lat=48.8592112&lon=2.338452");
        try(BufferedReader br = new BufferedReader(new InputStreamReader(url.openStream()))) {
            JSONArray fs= new JSONObject(br.readLine()).getJSONArray("features");
            for(int i=0; i != fs.length(); ++i){
                JSONObject f= fs.getJSONObject(i);
               if(f.getJSONObject("properties").getString("city").equals("Paris")){
                   if(f.getJSONObject("properties").getDouble("score")> score){
                       score= f.getJSONObject("properties").getDouble("score");
                    longitude=f.getJSONObject("geometry").getJSONArray("coordinates").getDouble(0);   
                   latitude=f.getJSONObject("geometry").getJSONArray("coordinates").getDouble(1);   
                   }
                }
                
            }
System.out.println("latitude: "+latitude+", longitude: "+longitude);
            //JSONArray res = new JSONObject(resp).getJSONArray("features").getJSONObject(0).getJSONObject("geometry").getJSONArray("coordinates");
    //System.out.println(res);//longitude , latitude
        }


latitude: 48.867413, longitude: 2.347572


null

In [3]:
package test.beaker;

import java.util.Date;
import java.text.SimpleDateFormat;

public class BeakerTest {
  private Date _date;
  private SimpleDateFormat sdf = new SimpleDateFormat("yyyy-MM-dd'T'HH:mmZ");
  
  public BeakerTest() {
    _date = new Date();
  }

  public String getDateTxt() {
    return "Today:" + sdf.format(_date);
  }

  public String getDateUpperCaseTxt() {
    return getDateTxt().toUpperCase();
  }

}


test.beaker.BeakerTest

In [15]:
import java.io.FileReader;
import java.io.BufferedReader;

import java.io.FileNotFoundException;
import java.io.IOException;

import java.io.InputStreamReader;
import java.net.URL;
import java.net.URLEncoder;

import org.json.JSONObject;
import org.json.JSONArray;

import java.util.Collection;
import java.util.List;
import java.util.ArrayList;
import java.util.Map;
import java.util.HashMap;
import java.util.Scanner;

public class GeoPoint{
    public final double latitude;
    public final double longitude;
    public final Map<GeoPoint, Integer> neighborToDistance;
    
    public static List<GeoPoint> data= new ArrayList<GeoPoint>();
    
    public GeoPoint(double latitude, double longitude){
        this.latitude= latitude;
        this.longitude= longitude;
	neighborToDistance= new HashMap<GeoPoint, Integer>();
    }

    public double distanceTo(GeoPoint other){
        return distanceTo(other.latitude, other.longitude);
    }
   public double distanceTo(double latitude, double longitude){
       double earthRadius= 6371e3;// in meters
        double phi1=Math.toRadians(this.latitude);
        double phi2=Math.toRadians(latitude);
        double deltaPhi=Math.toRadians(latitude- this.latitude);
        double deltaLambda= Math.toRadians(longitude- this.longitude);
        double a= Math.sin(deltaPhi/2) * Math.sin(deltaPhi/2) +
            Math.cos(phi1) * Math.cos(phi2) * Math.sin(deltaLambda/2) * Math.sin(deltaLambda/2);
        double c= 2* Math.atan2(Math.sqrt(a), Math.sqrt(1-a));
        return earthRadius * c;
    }
    public String toString(){
        return "GeoPoint: "+super.toString()+"{latitude: "+latitude+", longitude: " + longitude+", nb of neighbors"+neighborToDistance.size()+"}";
    }
    public static GeoPoint closerTo(double latitude, double longitude, double deltaD, Collection<GeoPoint> points){
        GeoPoint res=null;
        double minD= Double.POSITIVE_INFINITY;
        for(GeoPoint point : points){
            double currentD= point.distanceTo(latitude, longitude);
            if((currentD < minD) && (currentD > deltaD) ){
                minD= currentD;
                res= point;
            }
        }
        return res;
    }
    public static GeoPoint findByName(String name){
	    GeoPoint res= null;
        try{
            
            URL url= new URL("https://api-adresse.data.gouv.fr/search/?q="+URLEncoder.encode(name, "UTF-8")+"&postcode=75000");
            try(BufferedReader br = new BufferedReader(new InputStreamReader(url.openStream()))) {
                JSONArray coords = new JSONObject(br.readLine()).getJSONArray("features").getJSONObject(0).getJSONObject("geometry").getJSONArray("coordinates");
                res= new GeoPoint(coords.getDouble(0), coords.getDouble(1));
            }catch(Exception e){
                System.err.println(e);
            }
        }catch(Exception e){
            System.err.println(e);
        }  
    return res; // should throw
    }

    public static GeoPoint closerTo(String name, Collection<GeoPoint> data){
        return closerTo(findByName(name), data);
    }
    public static GeoPoint closerTo(GeoPoint ref, Collection<GeoPoint> data){
        return closerTo(ref.latitude, ref.longitude, 1.e-50, data);
    }
public static List<GeoPoint> read(String dataUrl) throws IOException {
        List<GeoPoint> res= new ArrayList<GeoPoint>();
        URL url= new URL(dataUrl);
        try(BufferedReader br = new BufferedReader(new InputStreamReader(url.openStream()))) {
            String[] headers= br.readLine().split(" ");
            int nbVertices= Integer.parseInt(headers[0]);
            int nbEdges= Integer.parseInt(headers[1]);
            for(int i=0; i != nbVertices; ++i){
                String[] coords= br.readLine().split(" ");//latitude, longitude
                res.add(new GeoPoint(Double.parseDouble(coords[0]), Double.parseDouble(coords[1])));
            }
            for(int i=0; i != nbEdges; ++i){
                String[] data= br.readLine().split(" ");//start end nbDirs Duration Length
                GeoPoint start= res.get(Integer.parseInt(data[0]));
                GeoPoint end= res.get(Integer.parseInt(data[1]));
                Integer length= Integer.parseInt(data[4]);
                start.neighborToDistance.put(end, length);
                if(data[2].equals("2")){
                    end.neighborToDistance.put(start, length);
                }
            }
        }catch (Exception e){
            System.err.println(e);
        }
        return res;
    }
    public static void main(String[] args){
        try{
            String url="https://raw.githubusercontent.com/jilljenn/tryalgo/master/examples/paris.txt";
            data= read(url);
        }catch(FileNotFoundException e){
            System.err.println(e);
        }
        catch(IOException e){
            System.err.println(e);
        }
    }
    
}

com.twosigma.beaker.javash.bkr750e75a1.GeoPoint

In [14]:
String[] args={};
GeoPoint.main(args);

[GeoPoint: com.twosigma.beaker.javash.bkr750e75a1.GeoPoint@769a232{latitude: 48.8351503, longitude: 2.3077904, nb of neighbors3}, GeoPoint: com.twosigma.beaker.javash.bkr750e75a1.GeoPoint@7cecdb39{latitude: 48.8317022, longitude: 2.3579381, nb of neighbors2}, GeoPoint: com.twosigma.beaker.javash.bkr750e75a1.GeoPoint@2e1b4b87{latitude: 48.8812439, longitude: 2.2917141, nb of neighbors1}, GeoPoint: com.twosigma.beaker.javash.bkr750e75a1.GeoPoint@3cca6971{latitude: 48.8558991, longitude: 2.3550941, nb of neighbors1}, GeoPoint: com.twosigma.beaker.javash.bkr750e75a1.GeoPoint@24f68858{latitude: 48.8406029, longitude: 2.3993259, nb of neighbors3}, GeoPoint: com.twosigma.beaker.javash.bkr750e75a1.GeoPoint@4b99bdfa{latitude: 48.8587301, longitude: 2.4106012000000003, nb of neighbors3}, GeoPoint: com.twosigma.beaker.javash.bkr750e75a1.GeoPoint@66f46c9e{latitude: 48.879117900000004, longitude: 2.3369039000000003, nb of neighbors3}, GeoPoint: com.twosigma.beaker.javash.bkr750e75a1.GeoPoint@735fed

ghbors2}, GeoPoint: com.twosigma.beaker.javash.bkr750e75a1.GeoPoint@76b75eca{latitude: 48.8718031, longitude: 2.2935687000000002, nb of neighbors3}, GeoPoint: com.twosigma.beaker.javash.bkr750e75a1.GeoPoint@2f2f1508{latitude: 48.859518, longitude: 2.3540808, nb of neighbors1}, GeoPoint: com.twosigma.beaker.javash.bkr750e75a1.GeoPoint@26cdd7c6{latitude: 48.8252916, longitude: 2.3626607, nb of neighbors1}, GeoPoint: com.twosigma.beaker.javash.bkr750e75a1.GeoPoint@4c17db8d{latitude: 48.837623300000004, longitude: 2.3223905, nb of neighbors3}, GeoPoint: com.twosigma.beaker.javash.bkr750e75a1.GeoPoint@45d6d448{latitude: 48.831445800000004, longitude: 2.3807413, nb of neighbors2}, GeoPoint: com.twosigma.beaker.javash.bkr750e75a1.GeoPoint@24bb3c8{latitude: 48.875665500000004, longitude: 2.3196156, nb of neighbors3}, GeoPoint: com.twosigma.beaker.javash.bkr750e75a1.GeoPoint@5f10f716{latitude: 48.867839200000006, longitude: 2.3139027000000003, nb of neighbors3}, GeoPoint: com.twosigma.beaker.ja

.javash.bkr750e75a1.GeoPoint@137bf00d{latitude: 48.8687618, longitude: 2.3110097, nb of neighbors1}, GeoPoint: com.twosigma.beaker.javash.bkr750e75a1.GeoPoint@528b6347{latitude: 48.875008, longitude: 2.3574993, nb of neighbors3}, GeoPoint: com.twosigma.beaker.javash.bkr750e75a1.GeoPoint@5edd1e2{latitude: 48.847566500000006, longitude: 2.3227914000000003, nb of neighbors2}, GeoPoint: com.twosigma.beaker.javash.bkr750e75a1.GeoPoint@4247acd4{latitude: 48.818424900000004, longitude: 2.3699035, nb of neighbors2}, GeoPoint: com.twosigma.beaker.javash.bkr750e75a1.GeoPoint@148e1482{latitude: 48.825801600000005, longitude: 2.371827, nb of neighbors4}, GeoPoint: com.twosigma.beaker.javash.bkr750e75a1.GeoPoint@3aec6987{latitude: 48.829765300000005, longitude: 2.3570651000000002, nb of neighbors3}, GeoPoint: com.twosigma.beaker.javash.bkr750e75a1.GeoPoint@50e29e71{latitude: 48.853041600000005, longitude: 2.2618851, nb of neighbors4}, GeoPoint: com.twosigma.beaker.javash.bkr750e75a1.GeoPoint@6b1e6d

38355, nb of neighbors1}, GeoPoint: com.twosigma.beaker.javash.bkr750e75a1.GeoPoint@1887dd1b{latitude: 48.888485900000006, longitude: 2.3355041, nb of neighbors2}, GeoPoint: com.twosigma.beaker.javash.bkr750e75a1.GeoPoint@7ed6833d{latitude: 48.8920808, longitude: 2.3462443, nb of neighbors2}, GeoPoint: com.twosigma.beaker.javash.bkr750e75a1.GeoPoint@512ffee7{latitude: 48.8589384, longitude: 2.4058751000000003, nb of neighbors3}, GeoPoint: com.twosigma.beaker.javash.bkr750e75a1.GeoPoint@c04a324{latitude: 48.836274700000004, longitude: 2.2570944, nb of neighbors1}, GeoPoint: com.twosigma.beaker.javash.bkr750e75a1.GeoPoint@3a2d0cc7{latitude: 48.8845478, longitude: 2.3215339, nb of neighbors2}, GeoPoint: com.twosigma.beaker.javash.bkr750e75a1.GeoPoint@4295a27c{latitude: 48.8450283, longitude: 2.2976957000000002, nb of neighbors2}, GeoPoint: com.twosigma.beaker.javash.bkr750e75a1.GeoPoint@30f04796{latitude: 48.848349600000006, longitude: 2.3760104, nb of neighbors3}, GeoPoint: com.twosigma.

0e75a1.GeoPoint@6a522e83{latitude: 48.837184400000005, longitude: 2.3001202000000003, nb of neighbors2}, GeoPoint: com.twosigma.beaker.javash.bkr750e75a1.GeoPoint@7d345a27{latitude: 48.8574357, longitude: 2.3796886, nb of neighbors2}, GeoPoint: com.twosigma.beaker.javash.bkr750e75a1.GeoPoint@4a9aa40a{latitude: 48.859144300000004, longitude: 2.3794671000000003, nb of neighbors3}, GeoPoint: com.twosigma.beaker.javash.bkr750e75a1.GeoPoint@68625236{latitude: 48.850145000000005, longitude: 2.3483419000000003, nb of neighbors1}, GeoPoint: com.twosigma.beaker.javash.bkr750e75a1.GeoPoint@2bf0d90c{latitude: 48.8871708, longitude: 2.3495686, nb of neighbors3}, GeoPoint: com.twosigma.beaker.javash.bkr750e75a1.GeoPoint@7d9cec90{latitude: 48.8294447, longitude: 2.3688264, nb of neighbors2}, GeoPoint: com.twosigma.beaker.javash.bkr750e75a1.GeoPoint@7b00de8e{latitude: 48.87276790000001, longitude: 2.397056, nb of neighbors2}, GeoPoint: com.twosigma.beaker.javash.bkr750e75a1.GeoPoint@39935e8c{latitude

...

9441000000003, nb of neighbors2}, GeoPoint: com.twosigma.beaker.javash.bkr750e75a1.GeoPoint@5597ffbb{latitude: 48.885247, longitude: 2.3033805000000003, nb of neighbors6}, GeoPoint: com.twosigma.beaker.javash.bkr750e75a1.GeoPoint@3e0c5529{latitude: 48.8478567, longitude: 2.4151291, nb of neighbors1}, GeoPoint: com.twosigma.beaker.javash.bkr750e75a1.GeoPoint@ba6eb96{latitude: 48.8775874, longitude: 2.4082945000000002, nb of neighbors2}, GeoPoint: com.twosigma.beaker.javash.bkr750e75a1.GeoPoint@d252678{latitude: 48.8481625, longitude: 2.3974807, nb of neighbors2}, GeoPoint: com.twosigma.beaker.javash.bkr750e75a1.GeoPoint@3a0dc5f5{latitude: 48.889327200000004, longitude: 2.3370177, nb of neighbors3}, GeoPoint: com.twosigma.beaker.javash.bkr750e75a1.GeoPoint@2e10f226{latitude: 48.8280303, longitude: 2.3924407000000003, nb of neighbors2}, GeoPoint: com.twosigma.beaker.javash.bkr750e75a1.GeoPoint@3e6984d1{latitude: 48.8696671, longitude: 2.3264549000000003, nb of neighbors2}, GeoPoint: com.t

ker.javash.bkr750e75a1.GeoPoint@1fdb9ce8{latitude: 48.851677300000006, longitude: 2.3352858000000003, nb of neighbors2}, GeoPoint: com.twosigma.beaker.javash.bkr750e75a1.GeoPoint@6127fc76{latitude: 48.8535185, longitude: 2.3097623, nb of neighbors3}, GeoPoint: com.twosigma.beaker.javash.bkr750e75a1.GeoPoint@69457955{latitude: 48.843245800000005, longitude: 2.383671, nb of neighbors2}, GeoPoint: com.twosigma.beaker.javash.bkr750e75a1.GeoPoint@1c383c68{latitude: 48.8421499, longitude: 2.2816381000000003, nb of neighbors2}, GeoPoint: com.twosigma.beaker.javash.bkr750e75a1.GeoPoint@4503750d{latitude: 48.845419, longitude: 2.2558730000000002, nb of neighbors2}, GeoPoint: com.twosigma.beaker.javash.bkr750e75a1.GeoPoint@5f683586{latitude: 48.876739400000005, longitude: 2.2945361, nb of neighbors1}, GeoPoint: com.twosigma.beaker.javash.bkr750e75a1.GeoPoint@603fca70{latitude: 48.8869897, longitude: 2.3862934, nb of neighbors3}, GeoPoint: com.twosigma.beaker.javash.bkr750e75a1.GeoPoint@2c043740{

ude: 2.3600994, nb of neighbors2}, GeoPoint: com.twosigma.beaker.javash.bkr750e75a1.GeoPoint@42d710a0{latitude: 48.859864800000004, longitude: 2.3517474000000003, nb of neighbors3}, GeoPoint: com.twosigma.beaker.javash.bkr750e75a1.GeoPoint@51149dd3{latitude: 48.859753700000006, longitude: 2.3519682, nb of neighbors3}, GeoPoint: com.twosigma.beaker.javash.bkr750e75a1.GeoPoint@4edec161{latitude: 48.861238500000006, longitude: 2.3606569, nb of neighbors2}, GeoPoint: com.twosigma.beaker.javash.bkr750e75a1.GeoPoint@33815ca{latitude: 48.878907700000006, longitude: 2.3782625, nb of neighbors2}, GeoPoint: com.twosigma.beaker.javash.bkr750e75a1.GeoPoint@473bf8ee{latitude: 48.8859937, longitude: 2.3892805000000004, nb of neighbors2}, GeoPoint: com.twosigma.beaker.javash.bkr750e75a1.GeoPoint@664c4a60{latitude: 48.833441, longitude: 2.3657363, nb of neighbors1}, GeoPoint: com.twosigma.beaker.javash.bkr750e75a1.GeoPoint@417bb1fc{latitude: 48.865756100000006, longitude: 2.3568701, nb of neighbors1},

rs3}, GeoPoint: com.twosigma.beaker.javash.bkr750e75a1.GeoPoint@46a64cea{latitude: 48.8720669, longitude: 2.3034816, nb of neighbors1}, GeoPoint: com.twosigma.beaker.javash.bkr750e75a1.GeoPoint@54af8495{latitude: 48.840714500000004, longitude: 2.2682241000000003, nb of neighbors4}, GeoPoint: com.twosigma.beaker.javash.bkr750e75a1.GeoPoint@5574374e{latitude: 48.8675926, longitude: 2.3415172, nb of neighbors2}, GeoPoint: com.twosigma.beaker.javash.bkr750e75a1.GeoPoint@1aba5ac1{latitude: 48.852675500000004, longitude: 2.3387938000000004, nb of neighbors3}, GeoPoint: com.twosigma.beaker.javash.bkr750e75a1.GeoPoint@6d9e8213{latitude: 48.8585421, longitude: 2.3678391000000003, nb of neighbors3}, GeoPoint: com.twosigma.beaker.javash.bkr750e75a1.GeoPoint@38998f47{latitude: 48.8845286, longitude: 2.359327, nb of neighbors3}, GeoPoint: com.twosigma.beaker.javash.bkr750e75a1.GeoPoint@27df5534{latitude: 48.884465500000005, longitude: 2.3494898, nb of neighbors2}, GeoPoint: com.twosigma.beaker.java

null

In [21]:
double minLat= Double.MAX_VALUE;
double maxLat= -Double.MAX_VALUE;
double minLong= Double.MAX_VALUE;
double maxLong= -Double.MAX_VALUE;
double sumLat= 0.;
double sumLong= 0.;
for(GeoPoint p: GeoPoint.data){
    sumLat+= p.latitude;
    sumLong+= p.longitude;
    if(p.latitude < minLat){ minLat= p.latitude;}
    if(p.latitude > maxLat){ maxLat= p.latitude;}
    if(p.longitude < minLong){ minLong= p.longitude;}
    if(p.latitude > maxLong){ maxLong= p.longitude;}
}
System.out.println("lat:["+minLat+", "+maxLat+"], long:["+minLong+", "+maxLong+"]");
System.out.println("avg lat: "+(sumLat/GeoPoint.data.size())+", avg long: "+(sumLong/GeoPoint.data.size()));

lat:[48.8157828, 48.901557700000005], long:[2.2519666000000003, 2.3598348000000002]
avg lat: 48.859211258010134, avg long: 2.338452206106807


null

In [27]:
import java.util.Arrays;
import java.util.List;
//class Display{
//public Plot map;
//    Display(dou)
//}
Plot map = new Plot();
Rasters r= new Rasters();
r.setFilePath("Paris.png");
r.setOpacity(Arrays.asList(new Number[]{0.5}));
Object[] xs={2.2519666000000003};//2.2519666000000003, 2.415388
Number[] ys={48.901557700000005};//48.8157828, 48.901557700000005
List<Object> x=Arrays.asList(xs);
        List<Number> y=Arrays.asList(ys);
r.setX(x);
r.setY(y);
Number[] hs={48.901557700000005-48.8157828};
List<Number> h= Arrays.asList(hs);
r.setHeight(h);
Number[] ws={2.415388-2.2519666000000003};
List<Number> w= Arrays.asList(ws);
r.setWidth(w);
map.add(r);
Points p= new Points();
xs=new Object[]{2.3469};
ys= new Number[]{48.8589};
p.setX(Arrays.asList(xs));
p.setY(Arrays.asList(ys));
p.setSize(Arrays.asList(new Number[]{6}));
map.add(p);
//[2.3469,48.8589]
return map;

/*
Points p= new Points();
ArrayList<Object> pXs= new ArrayList<Object>();
ArrayList<Number> pYs= new ArrayList<Number>();
for(GeoPoint pt : GeoPoint.data){
    pXs.add(pt.longitude);
    pYs.add(pt.latitude);
}
p.setX(pXs);
p.setY(pYs);
p.setSize(Arrays.asList(new Number[]{6}));
//map.add(p);
*/
    //[2.3469,48.8589]

In [7]:
package test.beaker;

BeakerTest bt = new BeakerTest();
return bt.getDateTxt();

Today:2018-08-26T06:58+0000

In [ ]:
import java.util.List;
import java.util.ArrayList;
import com.twosigma.beakerx.chart.xychart.Plot;
import com.twosigma.beakerx.chart.xychart.plotitem.*;
import com.twosigma.beakerx.chart.Color;

Plot p = new Plot();

p.setTitle("this is a Java plot");

Bars b = new Bars();

List<Number> yList = new ArrayList<Number>();
yList.add(2);
yList.add(5);
yList.add(4);
yList.add(8);

b.setY(yList);
b.setColor(Color.blue);
b.setWidth(0.5);

p.add(b);
  
return p;

In [ ]:
package test.beaker;
interface DateGetter {
   public String getDateTxt();
}

In [ ]:
package test.beaker;
public class DG2 extends BeakerTest implements DateGetter {
}

## Using the classpath to load a jar

In [ ]:
%classpath add jar ../resources/jar/BeakerXClasspathTest.jar

In [ ]:
System.out.println("The Groovy working folder is :");
System.out.println(java.nio.file.Paths.get(".").toAbsolutePath().normalize().toString());
System.out.println("BeakerXClasspathTest.jar exists in this folder");

In [ ]:
import com.beaker.BeakerXClasspathTest;

BeakerXClasspathTest t = new BeakerXClasspathTest();
System.out.println(com.beaker.BeakerXClasspathTest.staticTest);
System.out.println(t.getObjectTest());